In [1]:
# pylint: disable=missing-module-docstring
%load_ext autoreload
%autoreload 1
%aimport simulation

from collections import Counter
import os
import pandas as pd
import plotly.express as px

from simulation.parameters import Param
from simulation.model import Model

In [2]:
# Path to the outputs folder
OUTPUT_DIR = "../outputs/"

## Running the model

In [3]:
# Run the model
model = Model(param=Param(), run_number=0)
model.run()

In [4]:
# Preview some of the results
model.audit_list[0:5]

[{'time': 1095, 'asu_occupancy': 7, 'rehab_occupancy': 12},
 {'time': 1096, 'asu_occupancy': 5, 'rehab_occupancy': 11},
 {'time': 1097, 'asu_occupancy': 5, 'rehab_occupancy': 10},
 {'time': 1098, 'asu_occupancy': 4, 'rehab_occupancy': 10},
 {'time': 1099, 'asu_occupancy': 5, 'rehab_occupancy': 10}]

## Figure 1

**Figure 1.** Simulation probability density function for occupancy of an acute stroke unit.

**Relevant functions...**

In [5]:
def get_occupancy_freq(unit):
    """
    Count the frequency each occupancy level was observed at within the
    interval audit for the specified unit.

    Parameters
    ----------
    unit: str
        Name of unit to plot ("asu", "rehab").

    Returns
    -------
    df: pd.DataFrame
        Dataframe with two columns: the number of beds ("beds") and the
        frequency this was observed in the audit ("freq").
    """
    # Get occupancy audit results
    occ_audit = pd.DataFrame(model.audit_list)

    # Get the frequency of each number of beds in the specified unit
    frequency = Counter(occ_audit[f"{unit}_occupancy"])

    # Fill in any gaps - find the maximum bed count, then create complete
    # dictionary include bed counts which were never observed
    min_bed_count = min(frequency.keys())
    max_bed_count = max(frequency.keys())
    complete_frequency = {i: frequency.get(i, 0)
                          for i in range(min_bed_count, max_bed_count + 1)}

    # Convert into a dataframe
    df = pd.DataFrame(
        complete_frequency.items(), columns=["beds", "freq"])

    # Add column with frequencies converted to percentages
    df["pct"] = df["freq"] / df["freq"].sum() * 100

    return df

In [6]:
def plot_occupancy_freq(df, unit, file, path=OUTPUT_DIR):
    """
    Plot the frequency at which each occupancy level was observed in the audit.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe output by `get_occupancy_freq()` containing the frequency
        each occupancy was observed at.
    unit: str
        Name of unit ("asu", "rehab")
    file: str
        Filename to save figure to (e.g. "figure.png").
    path: str
        Path to save file to (excluding filename).
    """
    # Create plot
    fig = px.bar(df, x="beds", y="pct", color_discrete_sequence=["black"])

    # Specify axis labels and theme
    if unit == "asu":
        unit_lab = "acute"
    elif unit == "rehab":
        unit_lab = "rehabilitation"
    fig.update_layout(
        xaxis_title=f"No. patients in {unit_lab} unit",
        yaxis_title="% observations",
        template="plotly_white",
        height=450,
        width=800
    )

    # Add box around figure, and set tick spacing to 1
    fig.update_xaxes(linecolor="black", mirror=True, dtick=1)
    fig.update_yaxes(linecolor="black", mirror=True, ticksuffix="%")

    # Show figure
    fig.show()

    # Save figure
    fig.write_image(os.path.join(path, file))

**Generate plots...**

(the article just includes a plot for the acute stroke unit).

In [7]:
# Acute stroke unit
asu_occupancy = get_occupancy_freq("asu")
plot_occupancy_freq(asu_occupancy, unit="asu",
                    file="occupancy_freq_asu.png")

# Rehabilitation unit
rehab_occupancy = get_occupancy_freq("rehab")
plot_occupancy_freq(rehab_occupancy, unit="rehab",
                    file="occupancy_freq_rehab.png")